In [6]:
#-*- coding:utf-8 -*-
from commonTool import *
from config import *

import sys


def runDashScript(scriptPathName, parameter):
    optionStr = ''
    for key in parameter:
        optionStr += ' ' + key + '=' + parameter[key]
        
    cmdStr = 'java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar ' + crawlegoPath + ' ' + scriptPathName + optionStr
    print(cmdStr, flush=True)
    retCode = os.system(cmdStr)
    
    return retCode


In [ ]:
# file: dailyPrice.py

# KRX에서 지정한 날짜의 주가를 크롤링해 날짜별로 저장
# beginDay[, endDay] 지정 가능. 없으면 조회 날짜의 주가 크롤링

def writeDailyPrice(dayNo):
    outputDirPath = outputRawPath + 'priceDaily' + os.path.sep + dayNo[0:4] + os.path.sep
    mkdir(outputDirPath)

    print('Day', dayNo, 'Output Folder:', outputDirPath, flush=True)
    writeDailyPriceFromKRX(dayNo, outputDirPath)
    

jobName = 'get daily price'
begin(jobName)

d = dt.datetime.today()

beginDay = d.strftime('%Y%m%d')
if len(sys.argv) >= 2 and len(sys.argv[1]) == 8:
    beginDay = sys.argv[1]
    d = dt.datetime.strptime(beginDay, '%Y%m%d').date()
    
endDay = beginDay
if len(sys.argv) >= 3 and len(sys.argv[2]) == 8:
    endDay = sys.argv[2]

print('range', beginDay, endDay)

while beginDay <= endDay:
    writeDailyPrice(beginDay)
    d = d + dt.timedelta(days=1)
    beginDay = d.strftime('%Y%m%d')

end(jobName)


In [ ]:
# file: adjPriceYear.py
# Naver에서 수정주가를 년단위로 조회하여 저장

def writeAdjustPrice(codes, year, outFile):
    noLine = 0
    for code in codes:
        sTick = tm.time()
        noLine += 1
        print(noLine, code,'processing', flush=True)
        
        beginDay = year + '0101'
        endDay = year + '1231'

        # resp = fetchMonthStockData(code, beginDay, endDay)
        resp = fetchDailyStockData(code, beginDay, endDay)

        writeStockData(code, outFile, resp)

        print(noLine, code,'done', tm.time() - sTick, flush=True)


jobName = 'get adjust price'
begin(jobName)

(codes, types, referCodes) = getCompanyCodeFromFile(codePathFile)
print('codes', len(codes), len(types), len(referCodes)) # 첫 번째, 두 번째 값 같아야 함.


d = dt.datetime.today()
year = d.strftime('%Y')

if len(sys.argv) >= 2 and len(sys.argv[1]) == 4:
    year = sys.argv[1]

print('Fetching Year', year, flush=True)


outputDirPath = outputRawPath + 'adjustPrice' + os.path.sep
mkdir(outputDirPath)

outputDirPath += 'adjDailyPrice-' + year + '.csv'

outFile = open(outputDirPath, 'w', encoding='utf-8')
outFile.write('종목코드, 날짜, 시가, 고가, 저가, 종가, 거래량, 외국인소진율\n');

writeAdjustPrice(codes, year, outFile)

outFile.close()

end(jobName)
    

In [7]:
# file: uploadPrice.py [year]

jobName = 'upload price info to db'
begin(jobName)

year = today()[0:4]

if len(sys.argv) >= 2 and len(sys.argv[1]) == 4:
    year = sys.argv[1]

scriptPathName = crawlegoScriptPath + 'UPLOAD-PRICE.xml'

parameter = {
    'YEAR': year,
    'ADJ_PATH': outputRawPath + 'adjustPrice' + os.path.sep + 'adjDailyPrice',
    'KRX_PATH': outputRawPath + 'priceDaily' + os.path.sep + year + os.path.sep,
    'DO_SERVER': '13.124.29.70'
}

retCode = runDashScript(scriptPathName, parameter)

end(jobName)

>>>>>>> upload price info to db started
java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\UPLOAD-PRICE.xml YEAR=2021 ADJ_PATH=D:\work\jupyter\krx\output\adjustPrice\adjDailyPrice KRX_PATH=D:\work\jupyter\krx\output\priceDaily\2021\ DO_SERVER=13.124.29.70


KeyboardInterrupt: 

In [8]:
today()

'2021-06-17'